# Optimizacija fazi kontrolera za vožnju automobila korišćenjem genetskog algoritma

## Uvod

Cilj ovog rada je bila optimizacija fazi kontrolera za vožnju korišćenjem genetskog algoritma. Kontroler je upravljao automobilom koji se kretao po otvorenom ili zatvorenom putu, u zavisnosti od zadatih opcija.  

U radu je prvo pokazano da je moguće konstruisati fazi kontroler koji će pravilno upravljati automobilom, a zatim je pokazano i da je moguće optimizovati nasumično izgenerisan fazi sistem uz pomoć genetskog algoritma tako da pravilno upravlja automobilom.  

Automobilom se upravlja pravilno ako se automobil ne sudara sa ivicama puta i ako se kreće po sredini puta, koliko god je to moguće.

Automobil je testiran na dve vrste puta. Prvi tip puta je krivudav put koji je izgenerisan sinusiodom, a drugi tip puta je bio zatvoren kružni put izgenerisan pomoću konveksnog omotača nasumično odabranog skupa koordinata.

## Fazi sistem

Fazi sistem se sastoji iz 3 ulaza i 2 izlaza. Ulazi su predstavljali udaljenost od kraja puta, a izlazi su predstavljali promenu ugla i brzine.

### Ulaz

Postoje 3 ulazne funkcije:
- `left_sensor`
- `right_sensor`
- `front_sensor`

Svaka od njih ima 4 iste funkcije pripadnosti:  
- `close`
- `midrange`
- `far`
- `very far`

One označavaju koliko smo udaljenosti od svake od ivica puta. 

### Izlaz

Postoje 2 izlazne funkcije:
- `velocity`
- `angle`

`velocity` označava promenu brzine, dok `angle` označava promenu ugla skretanja volana.

`velocity` ima 4 funkcije pripadnosti:
- `low`
- `middle`
- `high`
- `very high`

`angle` ima 5 funkcija pripadnosti:
- `hard right`
- `right`
- `forward`
- `left`
- `hard left`

Sve funkcije pripadnosti su trapeznog oblika, sem funkcija na kraju i na početku intervala, čiji je oblik najsličniji ramp funkciji. Takođe, sve funkcije su imaju interval iz kog smeju da uzimaju vrednosti. Funkcije se mogu lepo videti u sledećem rečniku, koji se koristi pri generisanju nasumičnog fazi sistema:


In [ ]:
ALL_FUZZY_FUNCS = {
    "left_sensor": {
        "name": "left_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "right_sensor": {
        "name": "right_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "front_sensor": {
        "name": "front_sensor",
        "left_boundary": 0,
        "right_boundary": 20,
        "mf_names": ["close", "midrange", "far", "very far"],
        "is_input": True,
    },
    "velocity": {
        "name": "velocity",
        "left_boundary": 1,
        "right_boundary": 20,
        "mf_names": ["low", "middle", "high", "very high"],
        "is_input": False,
    },
    "angle": {
        "name": "angle",
        "left_boundary": -45,
        "right_boundary": 45,
        "mf_names": ["hard right", "right", "forward", "left", "hard left"],
        "is_input": False,
    }
}

## Fazi sistem koji upravlja automobilom

U ovom poglavlju je opisan fazi sistem koji je napravljen bez upotrebe genetskog algoritma, da bi bilo pokazano da samo fazi logika može upravljati automobilom, a i da bi postojala nekakva referenca za rezultat genetskog algoritma.

Inicijalizacije ovog fazi sistema se nalaze u fajlu `fuzzy_default.py`:

In [ ]:
import fuzzy

def generate_default_system():
    left = {
        "close"    : MFInput("close", np.array([0, 16]), np.array([1, 0])),
        "midrange" : MFInput("midrange", np.array([10, 14, 18, 24]), np.array([0, 1, 1, 0])),
        "far"      : MFInput("far", np.array([20, 36, 44]), np.array([0, 1, 0])),
        "very far" : MFInput("very far", np.array([32, 100]), np.array([0, 1]))
    }

    right = {
        "close"    : MFInput("close", np.array([0, 16]), np.array([1, 0])),
        "midrange" : MFInput("midrange", np.array([10, 16, 20, 28]), np.array([0, 1, 1, 0])),
        "far"      : MFInput("far", np.array([24, 40, 45]), np.array([0, 1, 0])),
        "very far" : MFInput("very far", np.array([35, 100]), np.array([0, 1]))
    }

    front = {
        "close"    : MFInput("close", np.array([0, 18]), np.array([1, 0])),
        "midrange" : MFInput("midrange", np.array([14, 20, 24, 28]), np.array([0, 1, 1, 0])),
        "far"      : MFInput("far", np.array([26, 34, 40]), np.array([0, 1, 0])),
        "very far" : MFInput("very far", np.array([35, 100]), np.array([0, 1])),
    }

    velocity = {
        "low"       : MFOutput("low", np.array([1, 2]), np.array([1, 0])),
        "middle"    : MFOutput("middle", np.array([1.5, 3, 3.5, 4]), np.array([0, 1, 1, 0])),
        "high"      : MFOutput("high", np.array([3.5, 4.5, 5.5]), np.array([0, 1, 0])),
        "very high" : MFOutput("very high", np.array([4, 8]), np.array([0, 1]))
    }

    angle = {
        "hard right" : MFOutput("hard right", np.array([25, 45]), np.array([1, 0])),
        "right"      : MFOutput("right", np.array([5, 22, 25]), np.array([0, 1, 0])),
        "forward"    : MFOutput("forward", np.array([-5, 0, 5]), np.array([0, 1, 0])),
        "left"       : MFOutput("left", np.array([-25, -22, -5]), np.array([0, 1, 0])),
        "hard left"  : MFOutput("hard left", np.array([-45, -25]), np.array([0, 1]))
    }

    left_sensor = FuzzyInput("left_sensor", np.array([
        left["close"],
        left["midrange"],
        left["far"],
        left["very far"]
    ]))

    right_sensor = FuzzyInput("right_sensor", np.array([
        right["close"],
        right["midrange"],
        right["far"],
        right["very far"]
    ]))

    front_sensor = FuzzyInput("front_sensor", np.array([
        front["close"],
        front["midrange"],
        front["far"],
        front["very far"] 
    ]))

    f_velocity = FuzzyOutput("velocity", np.array([
        velocity["low"],
        velocity["middle"],
        velocity["high"],
        velocity["very high"]
    ]))

    f_angle = FuzzyOutput("angle", np.array([
        angle["hard right"],
        angle["right"],
        angle["forward"],
        angle["left"],
        angle["hard left"]
    ]))

    return left_sensor, right_sensor, front_sensor, f_velocity, f_angle 

## Izgled Fazi klase TODO

O ovom poglavlju je opisana generalna struktura fuzzy.py fajla

##  Genetski algoritam

### Hromozom

Svaki hromozom sadrži fazi sisteme za ugao i brzinu, kao i fitnes. Početni fazi sistemi se generišu pomoću `fuzzy_generator.py` fajla i inicijalizacija sistema je takva da garantuje valjanost.

Valjanost podrazumeva da su funkcije pripadnosti za svaki od sistema unutar definisanih granica, kao i da ne postoji 
nepokriveni deo X ose, tj. funkcije sigurno pokrivaju sve moguće vrednosti.

Pravila korišćena za ugao:

In [ ]:
angle_rules = fuzzy.FuzzyRules(np.array([
    fuzzy.Rule(np.array([left["close"], right["midrange"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["midrange"], right["close"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["close"], right["far"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["far"], right["close"]]), angle["hard left"]),

    fuzzy.Rule(np.array([left["close"], front["close"]]), angle["hard right"]),
    fuzzy.Rule(np.array([right["close"], front["close"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["close"], front["midrange"]]), angle["right"]),
    fuzzy.Rule(np.array([right["close"], front["midrange"]]), angle["left"]),

    fuzzy.Rule(np.array([left["far"], right["midrange"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["midrange"], right["far"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["very far"], right["midrange"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["midrange"], right["very far"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["far"], right["close"]]), angle["left"]),
    fuzzy.Rule(np.array([left["close"], right["far"]]), angle["right"]),

    fuzzy.Rule(np.array([left["midrange"], right["midrange"]]), angle["forward"]),
    fuzzy.Rule(np.array([left["close"]]), angle["hard right"]),
    fuzzy.Rule(np.array([right["close"]]), angle["hard left"]),
]))

Pravila korišćena za brzinu:

In [ ]:
 velocity_rules = fuzzy.FuzzyRules(np.array([
        fuzzy.Rule(np.array([front["close"]]), velocity["low"]),
        fuzzy.Rule(np.array([front["midrange"]]), velocity["middle"]),
        fuzzy.Rule(np.array([front["far"]]), velocity["high"]),
        fuzzy.Rule(np.array([front["very far"]]), velocity["very high"])
    ]))

### Selekcija

Korišćena je turnirska selekcija u kombinaciji sa elitizmom. Elitizam je unapred uzimao određen broj najboljih jedinki, dok su ostale bile odabrane selekcijom:


In [ ]:
def create_group(population, group_size = TOURNAMENT_SIZE):
    ids = random.sample(range(0, population.size), group_size)
    return population[ids]

def select(population):
    # tournament selection
    group = create_group(population)
    result = group[0]
    for i in range(1, group.size):
        if group[i] < result:
            result = group[i]
    return result

### Ukrštanje

Korišćeno je uniformno ukrštanje po funkcijama pripadnosti za svako od izlaznih pravila fazi sistema. Odvojeno su ukrštani delovi fazi sistema koji opisuju ugao i delovi fazi sistema koji opisuju brzinu:

In [1]:
def crossover(p1, p2):
    c1 = copy.deepcopy(p1)
    c2 = copy.deepcopy(p2)
    for i in range(c1.FSAngle.inputs.size):
        fuzzy_output1 = c1.FSAngle.inputs[i]
        fuzzy_output2 = c2.FSAngle.inputs[i]

        for j in range(fuzzy_output1.inputs.size):
            mf_output1 = fuzzy_output1[j]
            mf_output2 = fuzzy_output2[j]

            r = random.random()
            if r < 0.5:
                swap(mf_output1, mf_output2)

    for i in range(c1.FSVelocity.inputs.size):
        fuzzy_output1 = c1.FSVelocity.inputs[i]
        fuzzy_output2 = c2.FSVelocity.inputs[i]

        for j in range(fuzzy_output1.inputs.size):
            mf_output1 = fuzzy_output1[j]
            mf_output2 = fuzzy_output2[j]

            r = random.random()
            if r < 0.5:
                swap(mf_output1, mf_output2)
                
    return c1, c2

###  Mutacija

Vršena je makro mutacija nad funkcijama pripadnosti koje određuju izlaze. Mutacija je podrazumevala generisanje potpuno nove funkcije pripadnosti uz poštovanje svih ograničenja koje funkcija mora da zadovolji, slično kao i pri generisanju inicijalne populacije.

Uzimajući u obzir performanse programa, postoje 2 verovatnoće za mutaciju:
- `MUTATION_RATE`
- `MUTATION_GENOM_RATE`

`MUTATION_RATE` označava da li uopšte želimo da mutiramo trenutni hromozom, odnosno da li uopšte ulazimo u logiku za mutaciju.`MUTATION_GENOM_RATE` označava verovatnoću da mutiramo jednu od funkcija pripadnosti nakon što je odlučeno da se trenutni hromozom mutira:

In [ ]:
def mutate(c, mutation_rate = MUTATION_RATE):
    r = random.random()
    if r > mutation_rate:
        return c

    for i in range(c.FSAngle.inputs.size):
        fuzzy_input = c.FSAngle.inputs[i]
        for j in range(fuzzy_input.inputs.size):
            mf_input = fuzzy_input[j]
            for k in range(fuzzy_input.inputs[j].size):
                
                r = random.random()
                if r < MUTATION_GENOM_RATE:
                    number_of_points = int(mf_input.size)
                    right_boundary = fuzzy_generator.ALL_FUZZY_FUNCS[fuzzy_input.name]["right_boundary"]
                    left_boundary = fuzzy_generator.ALL_FUZZY_FUNCS[fuzzy_input.name]["left_boundary"]

                    shift_value = random.randint(-MUTATION_SPAN, MUTATION_SPAN)

                    if number_of_points == 2 and mf_input.points[0][1] == 1:
                        new_value = mf_input.points[0][0] + shift_value
                        mf_input.points[0][0] = max(right_boundary, new_value)

                    elif number_of_points == 2 and mf_input.points[0][1] == 0:
                        new_value = mf_input.points[1][0] + shift_value
                        mf_input.points[1][0] = min(left_boundary, new_value)

                    else:
                        left = mf_input.points[0][0] + shift_value
                        right = mf_input.points[-1][0] + shift_value

                        if left < left_boundary:
                            left = left_boundary
                        
                        if right > right_boundary:
                            right = right_boundary
                        
                        if left > right:
                            swap(left, right_boundary)

                        if abs(left - right) <= 2:
                            right += 2 

                        mid_1, mid_2 = random.sample(range(left, right), 2)
                        
                        if mid_1 > mid_2:
                            swap(mid_1, mid_2)

                        new_xs = [left, mid_1, mid_2, right]

                        for t in range(4):
                            mf_input.points[t][0] = new_xs[t]
    return c

### Fitnes funkcija

Fitnes jedinke se računao tako što bi jedinka bila puštena da vozi po stazi sve dok jedan od sledećih uslova ne bude ispunjen:
- Jedinka je izletela sa puta
- Dostignut maksimalan broj iteracija (jedinka nikada ne izleće van puta, pa mora nekako da prekine vožnju)
- Jedinka se ne pomera više od X iteracija

Kako je cilj jedinke da vozi što duže i što je moguće više po sredini puta fintes funkcija je računata kao:  

`left_right/iteration + punishment`

`left_right` meri koliko je jedinka išla po sredini puta, `iteration` označava koliko dugo je vozila, a `punishment` je penalizacija za one jedinke koje su prestale da se kreću, ali i dalje nisu izašle van puta.

**Što je fitnes manji, jedinka je bolja**

In [ ]:
def evaluate(FSAngle, FSVelocity, road_matrix, memory):
    """
    Runs a single simulation, movement params are calculated based on the fuzzy systems FSAngle and FSVelocity
    """
    car = vehicle.Car(constants.CAR_POS_X, constants.CAR_POS_Y, constants.CAR_ANGLE)

    iteration = 0
    past_pos = car.center_position()

    dec = decoder.Decoder(FSAngle, FSVelocity, car)
    dt = TIME_STEP

    total_distance = 0
    punishment = 0
    left_right = 0

    while iteration <= MAX_ITERATIONS:
        
        car.left_sensor_input, car.front_sensor_input, car.right_sensor_input = get_sensors(car, road_matrix, memory)
        ds, drot = dec.get_movement_params()
        car.update(dt, ds, drot)

        iteration += 1 
        total_distance += ds
        left_right += abs(float(car.left_sensor_input) - float(car.right_sensor_input))

        if iteration % 100 == 0:
            past_x, past_y = past_pos
            curr_x, curr_y = car.center_position()
            if vehicle.distance(past_x, past_y, curr_x, curr_y) < MIN_DISTANCE:
                break
            else:
                past_pos = car.center_position()

        if car.is_idle(iteration) or car.is_collided2(road_matrix):
            punishment = 150
            break

    return left_right/iteration + punishment

## Generisanje puta

Za testiranje algoritma su korišćene 2 vrste puta:
- Konstantan krivudav put generisan sinusnom funkcijom
- Nasumično generisan put korišćenjem convex hull algoritma

Radi bržeg pokretanja programa, putevi se unapred mogu izgenerisati pokretanjem `display_matrix_generator.py` koji će snimiti matrice za obe vrste puta.

Generisanje krivudavog puta:

In [ ]:
def generate_sin_path():
    coords1 = []
    coords2 = []
    offset = 200
    num_of_points = 20

    for i in range(constants.SCREEN_WIDTH):
        coords1.append((i, math.sin(0.01*i) * 200 + 250))

    coords2 = lt.apply_translation(0, offset, coords1)
    coords2.reverse() #? in order to properly match coords for polygon

    polygon = coords1 + coords2
    
    is_closed = False
    return polygon, is_closed

## Rezultati

Generisanje konveksnog poligona:

In [ ]:
def generate_convex_polygon(): 
    #? Guarantees that we will have car in our convex polygon
    coords = [(constants.CAR_POS_X, constants.CAR_POS_Y + 10)]
    num_of_points = 20
    
    for i in range (num_of_points):
        x = random.randrange(constants.SCREEN_WIDTH)
        y = random.randrange(constants.SCREEN_HEIGHT)
        coords.append((x, y))

    coords1 = ch.gift_wrap(coords)

    scaling_factor = 0.6
    offset_x = 200
    offset_y = 150
    coords2 = lt.apply_scaling(scaling_factor, scaling_factor, coords1)
    coords2 = lt.apply_translation(offset_x, offset_y, coords2)

    polygon = [coords1, coords2]
    
    is_closed = True
    return polygon, is_closed

#### Krivudav put

![Krivudav put](img/sin_polygon_example.png)

#### Zatvoren put

![Zatvoren put](img/convex_polygon_example.png)


## Zaključak